# Introduction

Run this notebook to create an analytical application for the SBM charge - based of the Equity Delta Example. The input data will be stored in-memory and Atoti will perform the computation "on-the-fly" based on user query. You can filter, drill down and explore your data and the SBM metrics.

## About SBM

You will find below how the SBM logic can be described in python and injected into Atoti. We'll narrow down the use case to Equity Delta charge for simplicity. The other aggregation chains can be added in a similar manner.

Sensitivity-based mathod is one of the parametric market risk methodologies. It can be used to compute a VaR-like metric from sensitivities using pre-calibrated risk weights and correlations, through a sequence of nested variance covariance formulae. Due to its multiple benefits (see [2]), the method is widely used both internally by orgazations to manage market risk, as well as by Regulators for capital requirements purposes (FRTB) as well as margining (SIMM). 

The notebook assumes that the reader is familiar with the sensitivity-based method and terminology.

## What to expect

In this notebook, we'll read the required sensitivities and calculation parameters and create a chain of measures aggregating data:

sensitivities by trade -> weighted sensitivities by risk factor -> charges by bucket -> equity delta margin


## Input data

Input data is sourced in a CRIF-like format - see [3].

## References 

- [1]: Consolidated Basel Framework Chapter MAR21: https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101. 
- [2]: ISDA SIMM(TM): From Principles to Model Specification: https://www.isda.org/a/vAiDE/simm-from-principles-to-model-specification-4-mar-2016-v4-public.pdf
- [3]: ISDA Risk Data Standard: https://www.isda.org/a/owEDE/risk-data-standards-v1-36-public.pdf

# Input data files

These are the csv-files that we need to illustrate the Equity Delta aggregation:

- "bigger_data.csv" - sensitivities in a CRIF-like format
- "eq_delta_gamma.csv" - cross-bucket correlations, set for each pair of buckets
- "eq_delta_rho.csv" - risk factor correlations, set per bucket, i.e. all pairs of risk factors in a given bucket have the same correlation
- "eq_delta_rw.csv" - risk weights, set per bucket and per risk factor type (stored in Label2 crif field)

The example data files are stored in s3.

Let's start by launching atoti cube and web app:

In [1]:
import atoti as tt
import pandas as pd
from atoti.config import create_config

session = tt.create_session(
    config="./configuration.yaml", sampling_mode=tt.sampling.FULL
)

session.url

Welcome to atoti 0.4.3!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


'http://localhost:58723'

# Sensitivities datastore

In [2]:
risks_store = session.read_csv(
    "s3://data.atoti.io/notebooks/sbm/bigger_data.csv",
    keys=["TradeID", "PortfolioID", "RiskType", "Qualifier", "Label2"],
    store_name="Risks",
    types={"Bucket": tt.types.STRING},
)

cube = session.create_cube(risks_store)
lvl = cube.levels
m = cube.measures
h = cube.hierarchies

In [3]:
# So far we have loaded data only into the risks_store
risks_store.head()

,,,,,AmountUSD,Bucket
TradeID,PortfolioID,RiskType,Qualifier,Label2,,
0,Bigger_Portfolio,Risk_Equity,Wilmar International,REPO,-10332.09,1
52,Bigger_Portfolio,Risk_Equity,Zurich Insurance Group,REPO,-2148.42,8
56,Bigger_Portfolio,Risk_Equity,FAW Group,REPO,22932.05,9
63,Bigger_Portfolio,Risk_Equity,Lukoil,SPOT,111553.68,9
73,Bigger_Portfolio,Risk_Equity,Christian Dior,SPOT,-1961608.41,10


In [4]:
# creating a comparator to sort buckets as numbers:
# FIXME
# The "First term" formula gives an exception - unhashable type - if the comparator is enabled
# lvl["Bucket"].comparator = tt.comparator.first_members([str(i) for i in range(1, 13)])

At this point, a cube has been created, and we can start browsing the sensitivities:

In [5]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

In [6]:
# Let's create a first measure - to count distinct Qualifiers
m['Distinct Qualifiers Count'] = tt.agg.count_distinct(risks_store['Qualifier'])

In [7]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

# Risk weights datastore

In [8]:
# the risk weights data store is created from a csv:
eq_delta_risk_weights_store = session.read_csv(
    "s3://data.atoti.io/notebooks/sbm/parameters/eq_delta_rw.csv",
    keys=["Bucket", "Label2"],
    types={"Bucket": tt.types.STRING},
    store_name="RiskWeights",
)

# risks store is joined with the risk weights store
risks_store.join(eq_delta_risk_weights_store)

In [9]:
eq_delta_risk_weights_store.head(5)

,,RW
Bucket,Label2,
1,SPOT,0.55
2,SPOT,0.60
3,SPOT,0.45
4,SPOT,0.55
5,SPOT,0.30


The risk weights are now available in the cube as a measure.
The measure is defined for Bucket and Label2 - these hierarchies need to be present into the view.

In [10]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

In [11]:
# Let's put it into a folder and format as percentages:
m["RW.VALUE"].folder = "Parameters"
m["RW.VALUE"].formatter = "DOUBLE[#%]"

In [12]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

# Weighted sensitivities

In this section, we'll create a measure to compute weighted sensitivities defined in
[MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$WS_k=RW_k \cdot s_k$$

As the risk weights are defined for each Bucket and Label2, this is the level where sensitivities need to be multiplied by the risk weight.


In [13]:
# The input sensitivities are multplied by the risk weight for each Bucket and Label2,
# and then summed up to obtain weighted sensitivities:
m["WS"] = tt.agg.sum(
    m["AmountUSD.SUM"] * m["RW.VALUE"],
    scope=tt.scope.origin(lvl["Bucket"], lvl["Label2"]),
)

In [14]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

# Bucket-level aggregation

The weighted sensitivities by risk factor are rolled up into charges by bucket ("bucket-level charges"), using a variance-covariance-type of formula that can be found in the [MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$K_{b} =\sqrt{max \left( 0, \sum _{k\in b} WS_{k}^{2} +\sum _{k\in b}\sum
  _{l\in b, l\neq k}\rho_{kl}\cdot WS_k \cdot WS_l\right)}$$
  
In this section we'll create measures visualizing bucket-level charges. 

We will be leveraging the fact, that some of the risk factor pairs are correlated with the same correlation parameter, and provide a more computationally efficient calculation. 

## Risk factor correlations

The parameter $\rho_{kl}$ denotes correlation between two risk factors $k$ and $l$ in a pair of risk factors. The rules defining the equity delta correlations are set in [MAR21.78](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_78).

The rules can be summarised for each pair of risk factors as follows:

- Case 1: same name, different type: a single value -> 0.999
- Case 2: different name, same type: a single value depending on bucket, for example, 0.15
- Case 3: different name, different type: value depending on risk factor multiplier by 0.999, for example, 0.15 x 0.999

Stylized example:


| risk factors | name1-spot | name1-repo | name2-spot | name2-repo |
|------------|-------------|-------------|-------------|-------------|
| name1-spot | 1 |  |  |  |
| name1-repo | same_name_diff_type | 1 |  |  |
| name2-spot | rho_by_name | rho_by_name x type_multiplier | 1 |  |
| name2-repo | rho_by_name x type_multiplier | rho_by_name x type_multiplier | same_name_diff_type | 1 |

In [15]:
# Equity delta risk factor is defined as a combination of fields - "Qualifier" and "Label2", i.e. equity name and risk factor type.
# Creating variables:
same_risk_factor = 1.0
same_name_diff_type = 0.999
diff_type_multiplier = 0.999

In [16]:
# Creating a datastore holding correlations defined per bucket ([MAR21.78](2)):
eq_delta_rho = session.read_csv(
    "s3://data.atoti.io/notebooks/sbm/parameters/eq_delta_rho.csv",
    keys=["Bucket"],
    types={"Bucket": tt.types.STRING},
    store_name="RiskFactorCorrelations",
)
risks_store.join(eq_delta_rho)
eq_delta_rho.head(5)

,names_correlation
Bucket,
1,0.15
2,0.15
3,0.15
4,0.15
5,0.25


In [17]:
# Let's put it into a folder and format as percentages:
m["names_correlation.VALUE"].folder = "Parameters"
m["names_correlation.VALUE"].formatter = "DOUBLE[0.00]"

In [18]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

Since many of the risk factor pairs share the same correlation value, it is possible to optimize the variance-covariance aggregation. The efficiency of this calculation is critical when the data cardinality along the risk factor is high.

We decompose the formula into the three components:
1. contribution of pairs with both risk factors being spot - same correlation $\rho_{names}$ set per bucket,
2. contribution of pairs with both risk factors being repo - same correlation $\rho_{names}$ set per bucket,
3. contribution of pairs where one risk factor is spot, another one is repo - same correlation $\rho_{names} \cdot 0.999$. We'd need to account that for some of the pairs where one risk factor is spot, one is repo some the equity names will match and need to be correlated at 0.999


Let's start with the pairs where both risk factors are either spot or repo - cases 1. and 2. above. Since for any $k$ and $l$ the correlation $\rho_{kl}$ will be equal to correlation defined per bucket $\rho_{names}$, their contribution can be rewritten:

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \\ \sum_k WS_k^2 + \rho_{names} \cdot \left(\left( \sum_k WS_k \right)^2 - \sum_k WS_k^2  \right) = \\ (1-\rho_{names}) \cdot \sum_k WS_k^2 + \rho_{names} \cdot \left( \sum_k WS_k \right)^2  \label{reduced_formula} \tag{reduced_formula}$$

In [19]:
# This measure will display sum of WS_k squared by risk factor (Qualifier + Label2):
m["sum squares"] = tt.agg.square_sum(
    m["WS"], scope=tt.scope.origin(lvl["Qualifier"], lvl["Label2"])
)

In [20]:
reduced_formula = (1 - m["names_correlation.VALUE"]) * m["sum squares"] + m[
    "names_correlation.VALUE"
] * m["WS"] ** 2

# Total contribution of:
# - pairs having only spot risk factors,
# - pairs having only repo risk factors,
# is the sum of the reduced formula by Label2 members.

m["spot&repo pairs contribution"] = tt.agg.sum(
    reduced_formula, scope=tt.scope.origin(lvl["Label2"])
)

The contribution of the risk factors where one risk factor belongs to "SPOT" and the other belongs to "REPO" - case 3 above - can be rewritten:

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \\ \vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} \cdot \rho_{names} \cdot 0.999 + 0.999 \cdot (1 - \rho_{names}) \sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}} \label{spot_vs_repo} $$

where:
- J - is a matrix of ones,
- first term in the above formula performs aggregation of all sensitivities, as if they all are correlated at $\rho_{names}$,
- the second term is to correct the first term and to account for the fact that risk factors, where spot and repo risk factors have the same equity name, must be correlated at 0.999.


**First term**

The measure `sum WS_repo |J| WS_spot` will display $\vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} $.

In [21]:
# filtering for repo and spot risk factors:
m["WS_spot"] = tt.filter(m["WS"], lvl["Label2"] == "SPOT")
m["WS_repo"] = tt.filter(m["WS"], lvl["Label2"] == "REPO")

# Collect WS of the spot risk factors in a vector and show against every qualifier in a bucket:
weights_vector = tt.agg._vector(
    m["WS_spot"], scope=tt.scope.origin(lvl["Bucket"], lvl["Qualifier"])
)

# FIXME
# m["spot vector"] = tt.parent_value(
#     weights_vector, on=[h["Qualifier"], h["Label2"]]
# )
m["spot vector"] = weights_vector

# cross product of weighted sensitivities for the names, having both spot and repo sensitivities:
# Multiply vector WS of the spot risk factors by WS of each repo risk factor sum them up
repo_scalar_x_spot_vector = m["WS_repo"] * m["spot vector"]
m["sum WS_repo |J| WS_spot"] = tt.agg.stop(
    tt.agg.sum(
        tt.array.sum(repo_scalar_x_spot_vector),
        scope=tt.scope.origin(lvl["Bucket"], lvl["Qualifier"]),
    ),
    lvl["Bucket"],
)

**Second term**

The measure `sum WS repo and spot` will display $\sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}}$

In [22]:
# FIXME
# Below formula will multiply by null - might be a bad style??

In [23]:
m["WS_spot_parent"] = tt.agg.sum(m["WS_spot"], scope=tt.scope.origin(lvl["Label2"]))
m["WS_repo_parent"] = tt.agg.sum(m["WS_repo"], scope=tt.scope.origin(lvl["Label2"]))
m["sum WS repo and spot"] = tt.agg.stop(
    tt.agg.sum(
        m["WS_spot_parent"] * m["WS_repo_parent"],
        scope=tt.scope.origin(lvl["Qualifier"]),
    ),
    lvl["Bucket"],
)

**Combined result cross spot & repo pairs**

In [24]:
# final contribution of the pairs where one risk factor is spot, one is repo
m["cross repo spot contribution"] = (
    m["names_correlation.VALUE"] * m["sum WS_repo |J| WS_spot"] * 0.999
    + 0.999 * (1 - m["names_correlation.VALUE"]) * m["sum WS repo and spot"]
)

**Total Kb**

In [25]:
m["Kb"] = tt.where(
    lvl["Bucket"] == "11",
    tt.agg.sum(tt.abs(m["WS"]), scope=tt.scope.origin(lvl["Qualifier"], lvl["Label2"])),
    tt.sqrt(
        tt.max(
            0, m["spot&repo pairs contribution"] + 2 * m["cross repo spot contribution"]
        )
    ),
)

In [26]:
cube.query(
    m["Kb"],
    levels=lvl["Bucket"]
)

,Kb
Bucket,
1,"2,002,940.32"
10,"1,819,298.42"
11,"6,418,348.71"
12,"286,992.44"
2,"1,248,390.74"
3,"4,757.80"
4,"758,459.35"
5,"456,854.09"
7,"932,675.38"


# Cross-bucket aggregation

## Bucket correlations

In [27]:
eq_delta_buckets_correlations = session.read_csv(
    "s3://data.atoti.io/notebooks/sbm/parameters/eq_delta_gamma.csv",
    keys=["Bucket", "Other Bucket"],
    types={"Bucket": tt.types.STRING, "Other Bucket": tt.types.STRING},
    store_name="eq_delta_corr_outer",
)
risks_store.join(eq_delta_buckets_correlations)

In [28]:
cube.visualize()

Open the notebook in JupyterLab with the atoti extension installed and enabled to see this widget.

## Aggregating across buckets

In [29]:
# 21.4(5)(a):
m["WSb"] = m["WS"]
m["WSc"] = tt.at(m["WS"], {lvl["Bucket"]: lvl["Other Bucket"]})
m["sum Kb2 + sum sum WSb WSc gamma"] = tt.agg.square_sum(
    m["Kb"], scope=tt.scope.origin(lvl["Bucket"])
) + tt.agg.sum(
    m["WSb"] * m["WSc"] * m["gamma.VALUE"],
    scope=tt.scope.origin(lvl["Bucket"], lvl["Other Bucket"]),
)

# 21.4(5)(b):
m["Sb"] = tt.max(tt.min(m["WS"], m["Kb"]), -1.0 * m["Kb"])
m["Sc"] = tt.at(m["Sb"], {lvl["Bucket"]: lvl["Other Bucket"]})
m["sum Kb2 + sum sum Sb Sc gamma"] = tt.agg.square_sum(
    m["Kb"], scope=tt.scope.origin(lvl["Bucket"])
) + tt.agg.sum(
    m["Sb"] * m["Sc"] * m["gamma.VALUE"],
    scope=tt.scope.origin(lvl["Bucket"], lvl["Other Bucket"]),
)

m["Delta Margin"] = tt.where(
    m["sum Kb2 + sum sum WSb WSc gamma"] > 0,
    tt.sqrt(m["sum Kb2 + sum sum WSb WSc gamma"]),
    tt.sqrt(m["sum Kb2 + sum sum Sb Sc gamma"]),
)

In [30]:
cube.query(
    m["Delta Margin"]
)

,Delta Margin
0,"9,357,984.34"
